# BOSTON HOUSE PRICING

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')  #importamos librerias

ModuleNotFoundError: No module named 'pandas'

## Lets load the Boston House Pricing Dataset

In [ ]:
from sklearn.datasets import fetch_openml

boston = fetch_openml(name='boston')          #no me permitia importarlo, se hizo de esta manera
data = boston['data']
target = boston['target']



In [ ]:
boston.keys()

In [ ]:
## Lets check the description of the dataset
boston = fetch_openml(name='boston')


print(boston['DESCR'])

#observamos la descripcion

In [ ]:
print(boston.data)

In [ ]:
print(boston.data) #visualizmos la data

In [ ]:
print(boston.target)

In [ ]:
print(boston.feature_names)

## Preparing The Dataset

In [ ]:
dataset = pd.DataFrame(boston.data,columns=boston.feature_names)

In [ ]:
dataset.head()

In [ ]:
dataset['Price']=boston.target

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
## Summarizing The Stats of the data
dataset.describe()

In [ ]:
## Check the missing Values
dataset.isnull().sum()

In [ ]:
### EXploratory Data Analysis
## Correlation
dataset.corr()

In [ ]:
import seaborn as sns
sns.pairplot(dataset)

## Analyzing The Correlated Features

In [ ]:
dataset.corr()

In [ ]:
plt.scatter(dataset['CRIM'],dataset['Price'])
plt.xlabel("Crime Rate")                        #sugiere que los precios de las casas tienden a disminuir a medida que aumenta la tasa de criminalidad en el área.
plt.ylabel("Price")

In [ ]:
plt.scatter(dataset['RM'],dataset['Price'])
plt.xlabel("RM") # average number of rooms per dwelling
plt.ylabel("Price")   #a medida que el número promedio de habitaciones por vivienda aumenta, el precio medio de las viviendas también aumenta

In [ ]:
import seaborn as sns
sns.regplot(x="RM",y="Price",data=dataset)

In [ ]:
sns.regplot(x="LSTAT",y="Price",data=dataset) # % lower status of the population   A medida que el porcentaje de la población de "estatus más bajo" aumenta, el precio de la vivienda disminuye

In [ ]:
sns.barplot(x=dataset['RAD'], y=dataset['Price'])
plt.xlabel("RAD")
plt.ylabel("Price") #Esto sugiere que la accesibilidad a las autopistas radiales puede ser un
                      #factor importante en la determinación del valor de la propiedad en Boston.

In [ ]:
sns.regplot(x="PTRATIO",y="Price",data=dataset) #pupil-teacher ratio by town

In [ ]:
## Independent and Dependent features

X = dataset.iloc[:,:-1]   # todas las variables menos Precio, estas independientes
y = dataset.iloc[:,-1]    #Solo precio en esta caso la depnditente

In [ ]:
X.head() #mostramos los X

In [ ]:
y #show y

In [ ]:
##Train Test Split
from sklearn.model_selection import train_test_split   #en este caso entrenamos con train al 30 y el otro 70 para las pruevavs, con 42 para que salga el valor siempre
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
X_train.head(3) #mostrar los primeros 3

In [ ]:
X_test.head(3)

In [ ]:
## Standardize the dataset
from sklearn.preprocessing import StandardScaler   #escala los datos para que esten en la misma escala y distribucion
scaler=StandardScaler()

In [ ]:
X_train_scalado =scaler.fit_transform(X_train) # dar una media de cero y desviacion en 1 asi mejorar el rendimiento dle modelo

In [ ]:
X_test_scalado =scaler.transform(X_test) #dar normalizacion

In [ ]:
import pickle
pickle.dump(scaler,open('scaling.pkl','wb')) #guardar los datos

In [ ]:
X_train

In [ ]:
X_test

## Model Training

In [ ]:
from sklearn.linear_model import LinearRegression #importamos para un modelo lineal

In [ ]:
regression = LinearRegression()

In [ ]:
regression.fit(X_train,y_train) #en este caso podemos usarlo para hacer predcciones 

In [ ]:
## print the coefficients and the intercept
print(regression.coef_) #en este caso vemos las variables independientes si es positivo o negativo indicando asi su cambio

In [ ]:
print(regression.intercept_) #el valor de la casa cuando todo los otros valores son 0

In [ ]:
## on which parameters the model has been trained
regression.get_params()

In [ ]:
print(X_test.shape)

In [ ]:
print(X_train.shape)

In [ ]:
### Prediction With Test Data
reg_pred = regression.predict(X_test)

In [ ]:
reg_pred

## Assumptions

In [ ]:
## plot a scatter plot for the prediction
plt.scatter(y_test,reg_pred)

In [ ]:
## Residuals
residuals=y_test-reg_pred

In [ ]:
residuals

In [ ]:
## Plot this residuals 

sns.displot(residuals,kind="kde")

In [ ]:
## Scatter plot with respect to prediction and residuals
## uniform distribution
plt.scatter(reg_pred,residuals)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test,reg_pred))
print(mean_squared_error(y_test,reg_pred))
print(np.sqrt(mean_squared_error(y_test,reg_pred)))

## R square and adjusted R square


Formula

**R^2 = 1 - SSR/SST**


R^2	=	coefficient of determination
SSR	=	sum of squares of residuals
SST	=	total sum of squares


In [ ]:
from sklearn.metrics import r2_score
score=r2_score(y_test,reg_pred)
print(score)

**Adjusted R2 = 1 – [(1-R2)*(n-1)/(n-k-1)]**

where:

R2: The R2 of the model
n: The number of observations
k: The number of predictor variables

In [ ]:
#display adjusted R-squared
1 - (1-score)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

## New Data Prediction

In [ ]:
boston.data[0].reshape(1,-1)

In [ ]:
##transformation of new data
scaler.transform(boston.data[0].reshape(1,-1))

In [ ]:
regression.predict(scaler.transform(boston.data[0].reshape(1,-1)))

In [ ]:
dataset.head(1).to_json(orient = 'columns') 


## Pickling The Model file For Deployment

In [ ]:
import pickle

In [ ]:
pickle.dump(regression,open('regmodel.pkl','wb')) # guardamos el modelo

In [ ]:
pickled_model = pickle.load(open('regmodel.pkl','rb')) # cargando el modelo

In [ ]:
## Prediction
pickled_model.predict(scaler.transform(boston.data[0].reshape(1,-1)))

In [ ]:
import os

os.listdir('.')

In [ ]:
from google.colab import files
files.download('regmodel.pkl')